In [2]:
# from getopt import getopt
# import cloudpickle
# import pickle
# import sys
# import os
import numpy as np
import pandas as pd
# import seaborn as sns
# from scipy.special import logit, expit
# from scipy.stats import uniform, norm, bernoulli, betabinom
# from statsmodels.stats.proportion import proportions_ztest
# from matplotlib import pyplot as plt
# import pymc as pm
# import arviz as az
# from modeltools import mcmc_diagnostics, create_summary_stat
from downcast import downcast_df
# import jax
# from pymc.sampling_jax import sample_numpyro_nuts
# from time import time, sleep
# from datetime import timedelta

In [3]:
# Reading in data
raw_data = pd.read_csv("data/unit_level_ratings.csv",index_col = 0)
raw_data = raw_data.sort_values(by=["corpus", "model", "topic"])

# Creating identifier for each corpus, model, and topic
# Identifier is unique for topic 
corpus_ids = (raw_data.groupby(["corpus"], as_index=False)
    .agg({"intrusion":"count"})
    .drop(columns="intrusion"))
corpus_ids["corpus_id"] = corpus_ids.index

model_ids = (raw_data.groupby(["model"], as_index=False)
    .agg({"intrusion":"count"})
    .drop(columns="intrusion"))
model_ids["model_id"] = model_ids.index

cordel_ids = (raw_data.groupby(["corpus", "model"], as_index=False)
    .agg({"intrusion":"count"})
    .drop(columns="intrusion"))
cordel_ids["cordel_id"] = cordel_ids.index 

topic_ids = (raw_data.groupby(["corpus", "model", "topic"], as_index=False)
    .agg({"intrusion":"count"})
    .drop(columns="intrusion"))
topic_ids["topic_id"] = topic_ids["topic"].astype(np.int16)

rater_ids = (raw_data.groupby(["corpus", "rater"], as_index=False)
    .agg({"intrusion":"count"})
    .drop(columns="intrusion"))
rater_ids["rater_id"] = rater_ids.index 

d1 = pd.merge(raw_data, corpus_ids, on=["corpus"], how="left")
d2 = pd.merge(d1, model_ids, on=["model"], how="left")
d3 = pd.merge(d2, cordel_ids, on=["corpus","model"], how="left")
d4 = pd.merge(d3, rater_ids, on=["corpus", "rater"], how="left")
data = pd.merge(d4, topic_ids, on=["corpus", "model", "topic"], how="left")
data = data[["corpus_id", "model_id", "cordel_id", "topic_id", "rater_id", "intrusion", "confidence"]]
data, na_s = downcast_df(data)

In [4]:
a = (data.groupby(["cordel_id", "topic_id"])
         .agg({"intrusion":"sum"})
         .reset_index()
         .rename(columns={"intrusion":"sum"}))
b = (data.groupby(["cordel_id", "topic_id"])
         .agg({"intrusion":"count"})
         .reset_index()
         .rename(columns={"intrusion":"count"}))
c = pd.merge(a, b, on=["cordel_id", "topic_id"])
c["model_precision"] = c["sum"]/c["count"]

In [5]:
avg_model_prec = np.array(c.groupby("cordel_id").agg({"model_precision":"mean"})["model_precision"]).round(4)

In [8]:
print("Difference in average model precision of corpus/topic model pairs")
print(avg_model_prec.reshape(1,-1) - avg_model_prec.reshape(-1,1))

Difference in average model precision of corpus/topic model pairs
[[ 0.     -0.1219 -0.0506 -0.0079 -0.0625 -0.0395]
 [ 0.1219  0.      0.0713  0.114   0.0594  0.0824]
 [ 0.0506 -0.0713  0.      0.0427 -0.0119  0.0111]
 [ 0.0079 -0.114  -0.0427  0.     -0.0546 -0.0316]
 [ 0.0625 -0.0594  0.0119  0.0546  0.      0.023 ]
 [ 0.0395 -0.0824 -0.0111  0.0316 -0.023   0.    ]]


In [7]:
print("Average difference between topic models. Conducting simulations using 0.055 is perfectly valid")
print(sum([0.122, 0.050, 0.071, 0.055, 0.032, 0.023])/6)

0.05883333333333333
Average difference between topic models. Conducting simulations using 0.055 is perfectly valid


## Simulating test power